In [1]:
import sqlite3
from datetime import datetime, timedelta

In [2]:
conn = sqlite3.connect('oleg.db')
cur = conn.cursor()

### Количество сообщений

In [3]:
cur.execute('''
    select sender, count(sender)
    from meta
    group by sender''')
n_mes = cur.fetchall()

In [4]:
n_mes

[('Oleg', 26820), ('Val', 20642)]

### Количество знаков

In [5]:
cur.execute('''
    select sender, sum(length(text))
    from meta
    join texts on texts.id = meta.id
    group by sender''')
n_ch = cur.fetchall()

In [6]:
n_ch

[('Oleg', 453182), ('Val', 785954)]

### Среднее количество знаков

In [7]:
print('Oleg:', n_ch[0][1] / n_mes[0][1])

Oleg: 16.89716629381059


In [8]:
print('Val:', n_ch[1][1] / n_mes[1][1])

Val: 38.075477182443564


### Кто начинал диалоги + Кто отвечал быстрее

In [9]:
cur.execute('''
    select sender, datetime
    from meta''')
lines = cur.fetchall()

Будем считать, что если человек отвечает позже чем спустя три часа, то он начинает новый диалог. Если новый диалог не начинается, значит мы считаем время ожидания между сообщениями.

In [10]:
starts_oleg = 1
starts_val = 0
waited_oleg = timedelta()
waited_val = timedelta()

for i in range(1, len(lines)):
    
    prev = datetime.fromisoformat(lines[i-1][1])
    prev_per = lines[i-1][0]
    
    cur = datetime.fromisoformat(lines[i][1])
    d = timedelta(hours=3)
    cur_per = lines[i][0]
    
    if cur - prev >= d:
        if cur_per == 'Val':
            starts_val += 1
        else:
            starts_oleg += 1

    else:
        if prev_per != cur_per:
            dl = cur - prev
            if cur_per == 'Val':
                waited_val += dl
            else:
                waited_oleg += dl

Количество начатых диалогов:

In [11]:
starts_oleg, starts_val

(342, 1177)

Время ожидания:

In [12]:
waited_oleg, waited_val

(datetime.timedelta(days=56, seconds=18099),
 datetime.timedelta(days=38, seconds=28055))